- [обработка всплывающего окна](https://stackoverflow.com/questions/63299765/cant-locate-element-on-pop-up-window-using-selenium-on-python)
- [selenium](https://selenium-python.readthedocs.io/page-objects.html)

In [1]:
import re
from time import sleep
# from string import ascii_uppercase as alphabet
from queue import Queue
import datetime

from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [2]:
# alphabet_sample = set(alphabet) - set('BCDFGHIJKLMNOPQRTVXYZ')
# alphabet_sample

MAX_SEQ_COUNT_TO_DOWNLOAD = 10_000

with open('access_file.txt') as fin:
    LOGIN, PASSWORD = fin.readline().strip().split()

In [3]:
driver = webdriver.Firefox(executable_path='/home/mr/Downloads/geckodriver')


def click_elem(elem, time=0):
    if time == 5:
        return
    try:
        elem.click()
    except:
        sleep(5)
        click_elem(elem, time + 1)


def find_elem(css_selector, time=0):
    if time == 5:
        raise TimeoutError('cant find elem')
    try:
        return driver.find_element_by_css_selector(css_selector)
    except:
        sleep(5)
        return find_elem(css_selector)


def simple_click(css_selector):
    click_elem(find_elem(css_selector))


driver.get("https://www.gisaid.org/")
sleep(1)
simple_click(".Login")
sleep(1)
login_form = find_elem("#elogin")
password_form = find_elem("#epassword")
login_form.send_keys(LOGIN)
password_form.send_keys(PASSWORD)
sleep(.5)
simple_click(".form_button_submit")  # login
sleep(3)
click_elem(driver.find_elements_by_class_name(
    "sys-actionbar-action")[1])  # "Search" button on top
sleep(2)


def get_sequences_count(time=0, vaiting_point=407_835):
    if time == 5:
        raise TimeoutError('cant get Total viruses count')

    try:
        raw = find_elem("#c_qkezb7_rf_leftinfo").text  # Total: {}
    except:
        sleep(5)
        return get_sequences_count(time+1, vaiting_point=vaiting_point)

    if raw.startswith("Total"):
        n_virus_count = int("".join(re.split("\D+", raw)))

        if n_virus_count >= vaiting_point:
            sleep(5)
            return get_sequences_count(time+1, vaiting_point=vaiting_point)
        else:
            return n_virus_count
    else:
        raise Exception('cant get Total viruses count')


def reset():
    simple_click("#ce_qkezb7_ga .sys-form-button")  # "Reset" button
    sleep(3)

In [4]:
# [] interval
DATE_CSS = ".hasDatepicker"
FIRST_DATE = "2000-01-01"
SECOND_DATE = "2020-11-22"
TODAY = str(datetime.date.today())


def next_day_str(cur_day):
    cur = list(map(int, cur_day.split('-')))
    nxt = datetime.date(*cur) + datetime.timedelta(days=1)
    return str(nxt)


def set_date_interval(start_date=None):
    """return date interval with num of collected seqs less then 10_000"""
    start_date = stop_date = start_date or SECOND_DATE
    last_stop = stop_date
    reset()
    _, _, frm, to = driver.find_elements_by_css_selector(DATE_CSS)
    frm.send_keys(start_date)
    sleep(6)
    vaiting_point = get_sequences_count()
    
    last_cnt = 0
    while stop_date != TODAY:
        to.clear()
        to.send_keys(stop_date)
        sleep(4)
        n_viruses = get_sequences_count()
        if n_viruses > MAX_SEQ_COUNT_TO_DOWNLOAD:
            stop_date = last_stop
#             print(f"break because of {n_viruses} viruses")
            break
        last_cnt = n_viruses
        last_stop = stop_date
        stop_date = next_day_str(stop_date)
    return start_date, stop_date, last_cnt

In [5]:
%%time
set_date_interval(SECOND_DATE)

KeyboardInterrupt: 

In [8]:
intervals = []
# cur_date = next_day_str(SECOND_DATE)
cur_date = '2020-07-02'
cnt = 0
while True:
    _, stop_date, v_cnt = set_date_interval(cur_date)
    cnt += v_cnt
    print(f"from {repr(cur_date)} to {repr(stop_date)} - {v_cnt} viruses, total: {cnt}")
    intervals.append((cur_date, stop_date))
    cur_date = next_day_str(stop_date)

from '2020-07-02' to '2020-07-20' - 9799 viruses, total: 9799
from '2020-07-21' to '2020-08-06' - 9847 viruses, total: 19646
from '2020-08-07' to '2020-08-24' - 9864 viruses, total: 29510
from '2020-08-25' to '2020-09-08' - 9380 viruses, total: 38890
from '2020-09-09' to '2020-09-20' - 9091 viruses, total: 47981
from '2020-09-21' to '2020-09-30' - 9704 viruses, total: 57685
from '2020-10-01' to '2020-10-09' - 9925 viruses, total: 67610
from '2020-10-10' to '2020-10-18' - 9298 viruses, total: 76908
from '2020-10-19' to '2020-10-25' - 9074 viruses, total: 85982
from '2020-10-26' to '2020-11-03' - 9471 viruses, total: 95453
from '2020-11-04' to '2020-11-28' - 6672 viruses, total: 102125


KeyboardInterrupt: 

In [5]:
intervals = [
    ('2000-01-01', '2020-03-10'),
    ('2020-03-11', '2020-03-17'),
    ('2020-03-18', '2020-03-22'),
    ('2020-03-23', '2020-03-26'),
    ('2020-03-27', '2020-03-30'),
    ('2020-03-31', '2020-04-03'),
    ('2020-04-04', '2020-04-09'),
    ('2020-04-10', '2020-04-10'),
    ('2020-04-11', '2020-04-19'),
    ('2020-04-20', '2020-04-29'),
    ('2020-04-30', '2020-05-13'),
    ('2020-05-14', '2020-06-01'),
    ('2020-06-02', '2020-06-16'),
    ('2020-06-17', '2020-07-01'),
    ('2020-07-02', '2020-07-20'),
    ('2020-07-21', '2020-08-06'),
    ('2020-08-07', '2020-08-24'),
    ('2020-08-25', '2020-09-08'),
    ('2020-09-09', '2020-09-20'),
    ('2020-09-21', '2020-09-30'),
    ('2020-10-01', '2020-10-09'),
    ('2020-10-10', '2020-10-18'),
    ('2020-10-19', '2020-10-25'),
    ('2020-10-26', '2020-11-03'),
    ('2020-11-04', '2020-11-28'),
]

In [15]:
def high_quality_choise():
    high_quail_css = "td:nth-child(3) .sys-form-firow"
    galka_input_finder = ".sys-event-hook"
    
    a = driver.find_elements_by_css_selector(high_quail_css)
    galki = a[0].find_elements_by_css_selector(galka_input_finder)
    for e in galki:
        click_elem(e)
    sleep(3)

In [16]:
done_queries = []

q = Queue()
for i in intervals:
    q.put(i)

while not q.empty():
    reset()
    high_quality_choise()
    start_date, stop_date = q.get()
    print(f"from {repr(start_date)} to {repr(stop_date)}")
    frm, to, _, _ = driver.find_elements_by_css_selector(DATE_CSS)
    frm.clear()
    to.clear()
    frm.send_keys(start_date)
    to.send_keys(stop_date)
    sleep(5)
    n_virus_count = get_sequences_count()  # check sequences count
    print(f"Total: {n_virus_count} viruses")
    
    if n_virus_count < MAX_SEQ_COUNT_TO_DOWNLOAD:
        checkboxes = driver.find_element_by_class_name(
            "yui-dt-hd").find_elements_by_tag_name("input")
        if len(checkboxes) != 1:
            print("breaked because of doubled checkboxes")
            break
        sleep(1)
        click_elem(checkboxes[0])  # choose all seqs
        sleep(5)
        simple_click(".sys-form-button:nth-child(2)")  # 1st Download button
        sleep(5)
        try:
            iframe = driver.find_element_by_class_name("sys-overlay-style")  # iframe: pop-up window
        except:
            try:
                sleep(5)
                iframe = driver.find_element_by_class_name("sys-overlay-style")  # iframe: pop-up window
            except:
                raise Exception('cant find iframe')
            
        driver.switch_to.frame(iframe)
        download_button = driver.find_element_by_id(
            "c_qkezb7_vm").find_elements_by_tag_name("button")[1]  # 2nd Download button
        click_elem(download_button)
        sleep(10)
        driver.switch_to.default_content()
        done_queries.append((start_date, stop_date, n_virus_count))
        sleep(60)  # time to download
        print("Downloaded")

from '2000-01-01' to '2020-03-10'
Total: 7200 viruses
Downloaded
from '2020-03-11' to '2020-03-17'
Total: 7413 viruses
Downloaded
from '2020-03-18' to '2020-03-22'
Total: 6337 viruses
Downloaded
from '2020-03-23' to '2020-03-26'
Total: 5915 viruses
Downloaded
from '2020-03-27' to '2020-03-30'
Total: 5715 viruses
Downloaded
from '2020-03-31' to '2020-04-03'
Total: 5775 viruses
Downloaded
from '2020-04-04' to '2020-04-09'
Total: 6128 viruses
Downloaded
from '2020-04-10' to '2020-04-10'
Total: 883 viruses
Downloaded
from '2020-04-11' to '2020-04-19'
Total: 6507 viruses
Downloaded
from '2020-04-20' to '2020-04-29'
Total: 6402 viruses
Downloaded
from '2020-04-30' to '2020-05-13'
Total: 6469 viruses
Downloaded
from '2020-05-14' to '2020-06-01'
Total: 6451 viruses
Downloaded
from '2020-06-02' to '2020-06-16'
Total: 7040 viruses
Downloaded
from '2020-06-17' to '2020-07-01'
Total: 7599 viruses
Downloaded
from '2020-07-02' to '2020-07-20'
Total: 7695 viruses
Downloaded
from '2020-07-21' to '2020

In [17]:
driver.close()

### Attempts by names. Failed

In [95]:
done_queries = []

q = Queue()
for letter in alphabet:
    q.put(f"hCoV-19/{letter}")

simple_click("#ce_qkezb7_ga .sys-form-button")  # "Reset" button
sleep(3)

while not q.empty():
    query = q.get()
    virus_name_form = find_elem("#ce_qkezb7_fv_entry")  # Virus name form
    virus_name_form.clear()
    virus_name_form.send_keys(query)
    sleep(3)
    n_virus_count = get_sequences_count() # check sequences count
    #  download data
    if n_virus_count < MAX_SEQ_COUNT_TO_DOWNLOAD:
        checkboxes = driver.find_element_by_class_name(
            "yui-dt-hd").find_elements_by_tag_name("input")
        if len(checkboxes) != 1:
            print("breaked because of doubled checkboxes")
            break
        sleep(1)
        click_elem(checkboxes[0])  # choose all seqs
        sleep(3)
        simple_click(".sys-form-button:nth-child(2)")  # 1st Download button
        sleep(3)

        iframe = driver.find_element_by_class_name(
            "sys-overlay-style")  # iframe: pop-up window
        driver.switch_to.frame(iframe)
        download_button = driver.find_element_by_id(
            "c_qkezb7_vm").find_elements_by_tag_name("button")[1]  # 2nd Download button
        click_elem(download_button)
        sleep(8)
        driver.switch_to.default_content()

        done_queries.append((query, n_virus_count))
    else:
        suggestions = driver.find_elements_by_css_selector(
            "#ce_qkezb7_fv_acomplc li")  # collect suggested names
        for s in suggestions:
            if s.text.startswith("hCoV-19/"):
                q.put(s.text)
    sleep(3)
    simple_click("#ce_qkezb7_ga .sys-form-button")  # Reset button
    sleep(2)

TimeoutError: cant get Total viruses count

In [158]:
driver.close()